In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Veri setini yükle
train_data = pd.read_csv('/data/mnist_train.csv')
test_data = pd.read_csv('/data/mnist_test.csv')

# Veriyi küçült
train_data = train_data.sample(frac=0.06, random_state=1)
test_data = test_data.sample(frac=0.06, random_state=1)

# Bağımsız değişkenleri ve hedef değişkeni ayır
X_train = train_data.drop('label', axis=1).values
y_train = to_categorical(train_data['label'].values)

X_test = test_data.drop('label', axis=1).values
y_test = to_categorical(test_data['label'].values)

# Veriyi normalize et
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Giriş boyutlarını yeniden boyutlandır ve genişlet
X_train_reshaped = X_train.reshape(-1, 28, 28, 1)
X_test_reshaped = X_test.reshape(-1, 28, 28, 1)
X_train_resized = tf.image.resize(X_train_reshaped, (299, 299))
X_test_resized = tf.image.resize(X_test_reshaped, (299, 299))
X_train_resized_rgb = np.stack((X_train_resized[:, :, :, 0],) * 3, axis=-1)
X_test_resized_rgb = np.stack((X_test_resized[:, :, :, 0],) * 3, axis=-1)

# InceptionV3 modelini oluştur
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
model_inception = models.Sequential()
model_inception.add(base_model)
model_inception.add(layers.GlobalAveragePooling2D())
model_inception.add(layers.Dense(256, activation='relu'))
model_inception.add(layers.Dropout(0.5))
model_inception.add(layers.Dense(10, activation='softmax'))
model_inception.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğit
model_inception.fit(X_train_resized_rgb, y_train, epochs=5, batch_size=64, validation_split=0.2)

# Modeli değerlendir
y_pred_inception = model_inception.predict(X_test_resized_rgb)
y_pred_inception_classes = np.argmax(y_pred_inception, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy_inception = accuracy_score(y_test_classes, y_pred_inception_classes)
precision_inception = precision_score(y_test_classes, y_pred_inception_classes, average='weighted')
recall_inception = recall_score(y_test_classes, y_pred_inception_classes, average='weighted')
f1_inception = f1_score(y_test_classes, y_pred_inception_classes, average='weighted')

print(f"Accuracy: {accuracy_inception}")
print(f"Precision: {precision_inception}")
print(f"Recall: {recall_inception}")
print(f"F1 Score: {f1_inception}")


Epoch 1/5
45/45 [==============================] - 118s 1s/step - loss: 0.5309 - accuracy: 0.8448 - val_loss: 5.4973 - val_accuracy: 0.1014
Epoch 2/5
45/45 [==============================] - 44s 975ms/step - loss: 0.1495 - accuracy: 0.9580 - val_loss: 2.1579 - val_accuracy: 0.8333
Epoch 3/5
45/45 [==============================] - 45s 1s/step - loss: 0.1173 - accuracy: 0.9719 - val_loss: 1.0825 - val_accuracy: 0.8806
Epoch 4/5
45/45 [==============================] - 44s 969ms/step - loss: 0.1084 - accuracy: 0.9722 - val_loss: 0.4680 - val_accuracy: 0.9014
Epoch 5/5
19/19 [==============================] - 13s 343ms/step
Accuracy: 0.795
Precision: 0.9122910981156596
Recall: 0.795
F1 Score: 0.8111332470550774
